In [18]:
import torch


In [19]:
from torchvision.io.image import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

In [20]:
if torch.cuda.is_available():
  print("cuda GPU is available")
  DEVICE = 'cuda'
else:
  print("GPU is not available")
  DEVICE = 'cpu'

cuda GPU is available


In [21]:
DEVICE= 'cpu'   #Using CPU because the GPU memory on my laptop is low for this work. It will be slow in the CPU.

In [22]:
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.85).to(DEVICE)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [23]:
def draw_box(image, box): #inputs are numpy array, Function to display image with it's bounding boxes
  img = image.cpu().numpy()
  img = img.copy()
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  for i,b in enumerate(box):
    x1 =  int(b[0].item())
    y1 = int(b[1].item())
    x2 = int(b[2].item())
    y2 = int(b[3].item())
    
    cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
    cv2.putText(img, weights.meta["categories"][prediction['labels'][i].item()] , (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (26,269,0), 2)
    
    
  return img

In [24]:
preprocess = weights.transforms()

In [27]:
import cv2
cap = cv2.VideoCapture('traffic.mp4')

while True:
    ret, frame = cap.read()

    if ret:
        frame_cpy = frame.copy()
        frame_cpy = cv2.cvtColor(frame_cpy, cv2.COLOR_BGR2RGB)
        frame_cpy = torch.tensor(frame_cpy, dtype = torch.float32, device= DEVICE)/255.0
        frame_cpy = frame_cpy.permute(2,0,1)
        #frame_cpy = frame_cpy.reshape(1, frame_cpy)
        frame1 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        frame_cpy = preprocess(frame_cpy)
        
        batch = [frame_cpy]
        prediction = model(batch)[0]
        
        img = draw_box(torch.tensor(frame1), prediction['boxes'])        
        cv2.imshow('frame', img)
    
    if cv2.waitKey(1)== ord('q'):   #press q to quit 
        print("Exiting while loop")
        break    #out of while loop

      
cap.release() 
cv2.destroyAllWindows() 

Exiting while loop


In [8]:
cap.release() 
cv2.destroyAllWindows() 